# Description

It combines all coefficient values in one tissue (see `Settings` below) into a single dataframe for easier processing later.

# Modules loading

In [5]:
import pandas as pd
from pathlib import Path

from ccc import conf
from ccc.utils import get_upper_triag

# Settings

In [8]:
TOP_N_GENES = "all"
# gtex_v8_data_{tissue}-{gene_sel_strategy}-{corr_method}.pkl
DATA_DIR = Path("/mnt/data/proj_data/ccc-gpu/gene_expr/data/gtex_v8")
# whole blood by default, but this is a parameters cells that can be changed when running papermill
GTEX_TISSUE = "whole_blood"
GENE_SELECTION_STRATEGY = "var_pc_log2"

In [3]:
assert GTEX_TISSUE is not None, "Tissue not selected"

In [17]:
GENE_SELECTION_DIR = DATA_DIR / "gene_selection" / TOP_N_GENES
SIMILARITY_MATRICES_DIR = DATA_DIR / "similarity_matrices" / TOP_N_GENES

# Paths

In [19]:
INPUT_GENE_EXPR_DATA_FILE = (
    GENE_SELECTION_DIR / f"gtex_v8_data_{GTEX_TISSUE}-{GENE_SELECTION_STRATEGY}.pkl"
)
display(INPUT_GENE_EXPR_DATA_FILE)

assert INPUT_GENE_EXPR_DATA_FILE.exists()

PosixPath('/mnt/data/proj_data/ccc-gpu/gene_expr/data/gtex_v8/gene_selection/all/gtex_v8_data_whole_blood-var_pc_log2.pkl')

In [20]:
SIMILARITY_MATRIX_FILENAME_TEMPLATE = (
    "gtex_v8_data_{tissue}-{gene_sel_strategy}-{corr_method}.pkl"
)
INPUT_CORR_FILE_TEMPLATE = Path(
    SIMILARITY_MATRICES_DIR / SIMILARITY_MATRIX_FILENAME_TEMPLATE
)
display(INPUT_CORR_FILE_TEMPLATE)

PosixPath('/mnt/data/proj_data/ccc-gpu/gene_expr/data/gtex_v8/similarity_matrices/all/gtex_v8_data_{tissue}-{gene_sel_strategy}-{corr_method}.pkl')

In [21]:
OUTPUT_FILE = SIMILARITY_MATRICES_DIR / str(INPUT_CORR_FILE_TEMPLATE).format(
    tissue=GTEX_TISSUE,
    gene_sel_strategy=GENE_SELECTION_STRATEGY,
    corr_method="all",
)
display(OUTPUT_FILE)

PosixPath('/mnt/data/proj_data/ccc-gpu/gene_expr/data/gtex_v8/similarity_matrices/all/gtex_v8_data_whole_blood-var_pc_log2-all.pkl')

# Load data

## Gene Ensembl ID -> Symbol mapping

In [22]:
gene_map = pd.read_pickle(DATA_DIR / "gtex_gene_id_symbol_mappings.pkl")

In [23]:
gene_map = gene_map.set_index("gene_ens_id")["gene_symbol"].to_dict()

In [24]:
assert gene_map["ENSG00000145309.5"] == "CABS1"

## Gene expression

In [25]:
data = pd.read_pickle(INPUT_GENE_EXPR_DATA_FILE)

In [26]:
data.shape

(56200, 755)

In [27]:
data.head()

,GTEX-111YS-0006-SM-5NQBE,GTEX-1122O-0005-SM-5O99J,GTEX-1128S-0005-SM-5P9HI,GTEX-113IC-0006-SM-5NQ9C,GTEX-113JC-0006-SM-5O997,GTEX-117XS-0005-SM-5PNU6,GTEX-117YW-0005-SM-5NQ8Z,GTEX-1192W-0005-SM-5NQBQ,GTEX-1192X-0005-SM-5NQC3,GTEX-11DXW-0006-SM-5NQ7Y,...,GTEX-ZVE2-0006-SM-51MRW,GTEX-ZVP2-0005-SM-51MRK,GTEX-ZVT2-0005-SM-57WBW,GTEX-ZVT3-0006-SM-51MT9,GTEX-ZVT4-0006-SM-57WB8,GTEX-ZVTK-0006-SM-57WBK,GTEX-ZVZP-0006-SM-51MSW,GTEX-ZVZQ-0006-SM-51MR8,GTEX-ZXES-0005-SM-57WCB,GTEX-ZXG5-0005-SM-57WCN
gene_ens_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000169429.10,0.5623,0.8067,116.90000,4.047,211.0,58.110,68.38,249.50,5.095,295.9,...,39.960,0.1393,0.22380,245.0,513.6,1626.00,0.5633,515.70,1.19400,1163.00
ENSG00000135245.9,0.6529,1.3850,199.20000,2.266,116.7,192.300,161.50,263.50,23.540,251.9,...,114.300,1.8330,0.41150,149.0,935.3,233.60,0.8882,134.00,1.12000,295.70
ENSG00000163631.16,1.8480,0.2503,0.08429,1.251,1348.0,9.971,101.30,95.09,1.264,119.3,...,2.092,2.1100,0.03588,171.8,107.1,71.25,1.7720,309.60,0.07361,17.75
ENSG00000277632.1,1.6960,1.3450,235.10000,11.770,141.7,199.100,525.50,659.90,10.910,209.3,...,61.340,2.2500,0.72310,261.2,400.0,288.50,2.6960,287.50,3.32300,618.90
ENSG00000239839.6,185.2000,1.7790,694.30000,23.840,297.3,3122.000,2521.00,1504.00,80.060,652.0,...,1010.000,253.8000,94.52000,6083.0,2768.0,52.06,34.5700,17.36,352.30000,63.85


## CCC-GPU

In [29]:
clustermatch_df = pd.read_pickle(
    str(INPUT_CORR_FILE_TEMPLATE).format(
        tissue=GTEX_TISSUE,
        gene_sel_strategy=GENE_SELECTION_STRATEGY,
        corr_method="ccc_gpu",
    )
)

In [30]:
clustermatch_df.shape

(56200, 56200)

In [31]:
clustermatch_df.head()

gene_ens_id,ENSG00000169429.10,ENSG00000135245.9,ENSG00000163631.16,ENSG00000277632.1,ENSG00000239839.6,ENSG00000186652.9,ENSG00000129824.15,ENSG00000152463.14,ENSG00000123689.5,ENSG00000012223.12,...,ENSG00000199224.1,ENSG00000271798.1,ENSG00000229686.1,ENSG00000234273.1,ENSG00000271368.1,ENSG00000278424.1,ENSG00000199024.1,ENSG00000278281.1,ENSG00000232661.1,ENSG00000206780.1
gene_ens_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000169429.10,1.000000,0.780181,0.401717,0.789583,0.139359,0.524274,0.048667,0.365544,0.799041,0.343411,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000135245.9,0.780181,1.000000,0.429092,0.780181,0.159947,0.516615,0.042631,0.334136,0.752314,0.355974,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000163631.16,0.401717,0.429092,1.000000,0.408477,0.127681,0.401717,0.020316,0.232700,0.381776,0.261762,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000277632.1,0.789583,0.780181,0.408477,1.000000,0.151543,0.547588,0.055712,0.346530,0.761547,0.381776,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000239839.6,0.139359,0.159947,0.127681,0.151543,1.000000,0.272752,0.008065,0.100699,0.135411,0.318958,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
assert data.index.equals(clustermatch_df.index)

## Pearson

In [33]:
pearson_df = pd.read_pickle(
    str(INPUT_CORR_FILE_TEMPLATE).format(
        tissue=GTEX_TISSUE,
        gene_sel_strategy=GENE_SELECTION_STRATEGY,
        corr_method="pearson",
    )
)

In [34]:
pearson_df.shape

(56200, 56200)

In [35]:
pearson_df.head()

gene_ens_id,ENSG00000169429.10,ENSG00000135245.9,ENSG00000163631.16,ENSG00000277632.1,ENSG00000239839.6,ENSG00000186652.9,ENSG00000129824.15,ENSG00000152463.14,ENSG00000123689.5,ENSG00000012223.12,...,ENSG00000199224.1,ENSG00000271798.1,ENSG00000229686.1,ENSG00000234273.1,ENSG00000271368.1,ENSG00000278424.1,ENSG00000199024.1,ENSG00000278281.1,ENSG00000232661.1,ENSG00000206780.1
gene_ens_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000169429.10,1.000000,0.571672,0.126906,0.456538,0.127140,0.256347,0.300878,-0.268024,0.472277,0.355504,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000135245.9,0.571672,1.000000,0.110612,0.438929,0.209161,0.223671,0.203370,-0.325667,0.532544,0.406272,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000163631.16,0.126906,0.110612,1.000000,0.129248,0.021091,0.054572,0.072717,-0.131141,0.235882,0.098683,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000277632.1,0.456538,0.438929,0.129248,1.000000,0.199647,0.238356,0.366739,-0.361011,0.406964,0.416603,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000239839.6,0.127140,0.209161,0.021091,0.199647,1.000000,0.229287,0.022563,-0.183318,0.077410,0.541731,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
assert data.index.equals(pearson_df.index)

## Spearman

In [37]:
spearman_df = pd.read_pickle(
    str(INPUT_CORR_FILE_TEMPLATE).format(
        tissue=GTEX_TISSUE,
        gene_sel_strategy=GENE_SELECTION_STRATEGY,
        corr_method="spearman",
    )
)

In [38]:
spearman_df.shape

(56200, 56200)

In [39]:
spearman_df.head()

gene_ens_id,ENSG00000169429.10,ENSG00000135245.9,ENSG00000163631.16,ENSG00000277632.1,ENSG00000239839.6,ENSG00000186652.9,ENSG00000129824.15,ENSG00000152463.14,ENSG00000123689.5,ENSG00000012223.12,...,ENSG00000199224.1,ENSG00000271798.1,ENSG00000229686.1,ENSG00000234273.1,ENSG00000271368.1,ENSG00000278424.1,ENSG00000199024.1,ENSG00000278281.1,ENSG00000232661.1,ENSG00000206780.1
gene_ens_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000169429.10,1.000000,0.853836,0.709240,0.850371,0.379835,0.760102,0.243599,-0.694446,0.866183,0.657452,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000135245.9,0.853836,1.000000,0.675719,0.834496,0.422539,0.735840,0.233504,-0.574572,0.827193,0.626538,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000163631.16,0.709240,0.675719,1.000000,0.683018,0.388814,0.730922,0.177607,-0.558213,0.667917,0.613913,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000277632.1,0.850371,0.834496,0.683018,1.000000,0.402836,0.756369,0.281333,-0.667793,0.808998,0.659031,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000239839.6,0.379835,0.422539,0.388814,0.402836,1.000000,0.626519,0.045501,-0.341542,0.297492,0.691271,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
assert data.index.equals(spearman_df.index)

## Merge

In [25]:
# # make sure genes match
# clustermatch_df = clustermatch_df.loc[pearson_df.index, pearson_df.columns]

In [41]:
clustermatch_df = get_upper_triag(clustermatch_df)

In [42]:
clustermatch_df = clustermatch_df.unstack().rename_axis((None, None)).dropna()

: 

: 

: 

In [28]:
clustermatch_df.shape

(12497500,)

In [29]:
clustermatch_df.head()

ENSG00000135245.9   ENSG00000169429.10    0.780181
ENSG00000163631.16  ENSG00000169429.10    0.401717
                    ENSG00000135245.9     0.429092
ENSG00000277632.1   ENSG00000169429.10    0.789583
                    ENSG00000135245.9     0.780181
dtype: float64

In [30]:
pearson_df = get_upper_triag(pearson_df)

In [31]:
# make pearson abs
pearson_df = pearson_df.unstack().rename_axis((None, None)).dropna().abs()

In [32]:
pearson_df.shape

(12497500,)

In [33]:
pearson_df.head()

ENSG00000135245.9   ENSG00000169429.10    0.571672
ENSG00000163631.16  ENSG00000169429.10    0.126906
                    ENSG00000135245.9     0.110612
ENSG00000277632.1   ENSG00000169429.10    0.456538
                    ENSG00000135245.9     0.438929
dtype: float64

In [34]:
assert clustermatch_df.index.equals(pearson_df.index)

In [35]:
spearman_df = get_upper_triag(spearman_df)

In [36]:
# make spearman abs
spearman_df = spearman_df.unstack().rename_axis((None, None)).dropna().abs()

In [37]:
spearman_df.shape

(12497500,)

In [38]:
spearman_df.head()

ENSG00000135245.9   ENSG00000169429.10    0.853836
ENSG00000163631.16  ENSG00000169429.10    0.709240
                    ENSG00000135245.9     0.675719
ENSG00000277632.1   ENSG00000169429.10    0.850371
                    ENSG00000135245.9     0.834496
dtype: float64

In [39]:
assert clustermatch_df.index.equals(spearman_df.index)

In [40]:
df = pd.DataFrame(
    {
        "ccc": clustermatch_df,
        "pearson": pearson_df,
        "spearman": spearman_df,
    }
).sort_index()

In [41]:
assert not df.isna().any().any()

In [42]:
df.shape

(12497500, 3)

In [43]:
df.head()

ccc   pearson  spearman
ENSG00000000419.12 ENSG00000002834.17  0.418721  0.681847  0.786595
                   ENSG00000002919.14  0.405090  0.734699  0.816991
                   ENSG00000002933.7   0.007466  0.013825  0.004128
                   ENSG00000003402.19  0.391683  0.727347  0.803653
                   ENSG00000004478.7   0.099013  0.094147  0.231269

# Save

In [44]:
df.to_pickle(OUTPUT_FILE)